In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from glob import glob
import itertools
from functools import lru_cache

ModuleNotFoundError: No module named 'pandas'

In [2]:
!pip install numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 24.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [3]:
path_data = Path(f"./Challenge 3 Variation mesures sondes fixes")
network_state_path = path_data / f"Etats reseaux telecoms"

measure_df = pd.read_csv(path_data / f"Mesures_exposition_sondes_autonomes.csv", sep=";")
mse_df = pd.read_csv(path_data / f"Dates_mise-en-service_sondes_autonomes.csv", sep=",")

FileNotFoundError: [Errno 2] No such file or directory: 'Challenge 3 Variation mesures sondes fixes/Mesures_exposition_sondes_autonomes.csv'

In [ ]:
measure_df["date"]

# Enrichissement de `measure_df`

In [ ]:
measure_df["datetime"] = [datetime.strptime(d, "%d/%m/%Y %H:%M") for d in measure_df["date"]]
measure_df["weekday"] = [f.weekday() for f in measure_df["datetime"]]
measure_df["month"] = [f.month for f in measure_df["datetime"]]
measure_df["day"] = [f.day for f in measure_df["datetime"]]
measure_df["hour"] = [f.hour for f in measure_df["datetime"]]

In [ ]:
measure_df

In [ ]:
network_state_dict = {}
for path in network_state_path.glob("*.csv"):
    date = path.name.split("_Etat reseaux.csv")[0]
    network_state_dict[date] = pd.read_csv(path.absolute(), sep=";")

In [ ]:
print(f"les dates disponibles : {tuple(network_state_dict.keys())}")

In [ ]:
network_state_dict["2020_03_31"]

# Afficher une sonde

In [ ]:
sonde = f"Marseille_{1:0>2}"

In [ ]:
measure_df

In [ ]:
measure_df[measure_df["numero"]==sonde]

In [ ]:
timestamps = measure_df["date"]
print(len(timestamps))
print(len(set(timestamps)))
print(len(timestamps))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))
DF = pd.DataFrame()
DF['value'] = measure_df[measure_df["numero"]==sonde]["E_volt_par_metre"]
DF = DF.set_index(measure_df[measure_df["numero"]==sonde]["datetime"])
plt.plot(DF, 'r+')
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
dt = measure_df[measure_df["numero"]==sonde]["datetime"][0]
dt.weekday()

In [ ]:
measure_df["weekday"]==2

In [ ]:
((measure_df["numero"]==sonde) & (measure_df["weekday"]==2)).sum()

In [ ]:
measure_df[(measure_df["numero"]==sonde) & (measure_df["weekday"]==2)]

In [ ]:
xrange = tuple(range(7))
res = []
for x in xrange:
    res.append(measure_df[(measure_df["numero"]==sonde) & (measure_df["weekday"]==x)]["E_volt_par_metre"].mean())
plt.plot(xrange, res, "r+")
plt.title(f"evolution sonde {sonde} en fonction jour de la semaine")

In [ ]:
xrange = tuple(range(7))
res = []
for x in xrange:
    res.append(measure_df[(measure_df["weekday"]==x)]["E_volt_par_metre"].mean())
plt.plot(xrange, res, "r+")
plt.title(f"evolution moyenne toutes les sondes en fonction jour de la semaine")

In [ ]:
plt.title(f"Evolution moyenne sonde {sonde} sur les mois")
xrange=np.sort(measure_df["month"].unique()) # == range(1, 13)
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & (measure_df["month"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")

In [ ]:
plt.title(f"Evolution moyenne de toutes les sondes sur les mois")
xrange=np.sort(measure_df["month"].unique()) # == range(1, 13)
plt.plot(xrange, [measure_df[(measure_df["month"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")

In [ ]:
plt.title(f"Evolution moyenne sonde {sonde} sur les jours de la semaines")
xrange=np.sort(measure_df["day"].unique()) # == range(1, 31 + 1)
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & (measure_df["day"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")

In [ ]:
plt.title(f"Evolution moyenne de toutes les sondes sur les jours de la semaines")
xrange=np.sort(measure_df["day"].unique()) # == range(1, 31 + 1)
plt.plot(xrange, [measure_df[(measure_df["day"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")

In [ ]:
plt.title(f"Evolution moyenne sonde {sonde} sur les heures paires des jours")
xrange = tuple(range(0, 24, 2))
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & (measure_df["hour"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")
plt.show()
plt.title(f"Evolution moyenne de toutes les sondes sur les heures paires des jours")
plt.plot(xrange, [measure_df[(measure_df["hour"]==x)]["E_volt_par_metre"].mean() for x in xrange], "r+")

In [ ]:
xrange, np.sort(measure_df["hour"].unique()) 

In [ ]:
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks or blocks.

    >>> grouper('ABCDEFG', 3, 'x')
    ['ABC', 'DEF', 'Gxx']
    """
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [ ]:
tuple(grouper(range(24), 2))

In [ ]:
plt.title(f"Evolution moyenne sonde {sonde} sur les heures des jours (groupées par 2)")
bins = tuple(grouper(range(24), 2))
xrange = range(len(bins))
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & ((measure_df["hour"]==bin_[0]) | (measure_df["hour"]==bin_[1]))]["E_volt_par_metre"].mean() for bin_ in bins], "r+")
plt.show()
plt.title(f"Evolution moyenne de toutes les sondes sur les heures paires des jours (groupées par 2)")
plt.plot(xrange, [measure_df[((measure_df["hour"]==bin_[0]) | (measure_df["hour"]==bin_[1]))]["E_volt_par_metre"].mean() for bin_ in bins], "r+")
plt.xlabel()

In [ ]:
plt.figure(figsize=(15, 4))
plt.title(f"Evolution sur l'année au jour près pour la sonde {sonde}")
xrange = tuple(range(365*12))
@lru_cache(maxsize=128)
def getDayMonthFromidx(idx:int):
    datetime_ = datetime(2020, 1, 1) + idx * timedelta(hours=24) #idx indique le jour de l'année
    return datetime_.day, datetime_.month
day_month = [getDayMonthFromidx(i) for i in xrange]
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & (measure_df["month"]==month) & (measure_df["day"]==day)]["E_volt_par_metre"].mean() for day, month in day_month], "r+")
plt.show()
plt.title(f"Evolution moyenne de toutes les sondes sur les heures paires des jours")
plt.plot(xrange, [measure_df[(measure_df["month"]==month) & (measure_df["day"]==day)]["E_volt_par_metre"].mean() for day, month in day_month], "r+")

In [ ]:
plt.figure(figsize=(15, 4))
plt.title(f"Evolution sur l'année au jour près pour la sonde {sonde}")
xrange = tuple(range(365*12))
@lru_cache(maxsize=128)
def getDayMonthFromidx(idx:int):
    datetime_ = datetime(2020, 1, 1) + idx * timedelta(hours=24) #idx indique le jour de l'année
    return datetime_.day, datetime_.month
day_month = [getDayMonthFromidx(i) for i in xrange]
plt.plot(xrange, [measure_df[(measure_df["numero"]==sonde) & (measure_df["month"]==month) & (measure_df["day"]==day)]["E_volt_par_metre"].mean() for day, month in day_month], "r+")
plt.show()
plt.title(f"Evolution moyenne de toutes les sondes sur les heures paires des jours")
plt.plot(xrange, [measure_df[(measure_df["month"]==month) & (measure_df["day"]==day)]["E_volt_par_metre"].mean() for day, month in day_month], "r+")

In [ ]:
measure_df

In [ ]:
# network_state_df = 

In [ ]:
dfs = []
for date, df in network_state_dict.items():
    df["date"] = date
    dfs.append(df)
network_state_df = pd.concat(dfs)
network_state_df.head()

In [ ]:
antennas = list(set(network_state_df["EMETTEUR ID"]))
print(len(antennas), "émetteurs distincts")

sondes = list(set(measure_df["numero"]))
print(len(sondes), "sondes distinctes")

In [ ]:
datetimes = list(set(measure_df["datetime"]))
print(len(datetimes), "datetimes distincts")
trimesters = list(sorted(set(network_state_dict.keys()))
print(trimesters)

In [ ]:
activity_df = pd.DataFrame(columns=antennas, index=trimesters).fillna(0).astype(bool)
activity_df

In [ ]:
for trimester in trimesters:
    for antenna in antennas:
        # print(list(network_state_dict[trimester]["EMETTEUR ID"]))
        if antenna in list(network_state_dict[trimester]["EMETTEUR ID"]):
            activity_df[antenna][trimester] = True
activity_df

In [ ]:
activity_df_t = activity_df.transpose()
activity_df_t

In [ ]:
print("salut")